In [ ]:
import pandas as pd
import json
from pathlib import Path
import datetime
import src.std_utils as std_utils
import plotly.express as px
import pydeck as pdk

kDataPath = Path('data/agg')

def load_data():

    # Loads plants
    plants = std_utils.from_csv(kDataPath / 'plants.csv')
    plants_name = plants[['plant_code', 'plant_name']]

    # Loads data, merges with plants information (to get plants name)
    realtime = pd.merge(std_utils.from_csvs(
        kDataPath, '**/realtime.csv'), plants_name, on=['plant_code'])
    hourly = pd.merge(std_utils.from_csvs(
        kDataPath, '**/hourly.csv'), plants_name, on=['plant_code'])
    daily = pd.merge(std_utils.from_csvs(
        kDataPath, '**/daily.csv'), plants_name, on=['plant_code'])
    monthly = pd.merge(std_utils.from_csvs(
        kDataPath, '**/monthly.csv'), plants_name, on=['plant_code'])
    yearly = pd.merge(std_utils.from_csvs(
        kDataPath, '**/yearly.csv'), plants_name, on=['plant_code'])
    alarms = pd.merge(std_utils.from_csvs(
        kDataPath, '**/alarms.csv'), plants_name, on=['plant_code'])

    return {'plants': plants,
            'realtime': realtime,
            'hourly': hourly,
            'daily': daily,
            'monthly': monthly,
            'yearly': yearly,
            'alarms': alarms}

data = load_data()

# Data utilities

def latest_date(data):
    return data['realtime']['collect_time'].max().to_pydatetime()


def latest_data(data):
    return data[data.groupby(['plant_code'])['collect_time'].transform(
        max) == data['collect_time']]


def data_in_range(data, begin, end):
    return data.loc[(data['collect_time'] >= begin)
                    & (data['collect_time'] <= end)]


In [ ]:
def plot_heatlh(data):

    health = latest_data(data['realtime'])
    count = health['health_state'].value_counts()

    df = pd.DataFrame({'Quality of Service': count}).reset_index()

    fig = px.pie(df, df['Quality of Service'],
        color_discrete_map=std_utils.health_state_colormap(), labels=std_utils.descriptions())
    fig.update_layout(xaxis_title=None)

    return fig

plot_heatlh(data)


In [ ]:
def plot_qos(data):

    def qos_data(data):
        realtime = data['realtime']
        by_plant = realtime.groupby(['plant_name'])
        normalized = by_plant['health_state'].value_counts(normalize=True)
        count = by_plant['health_state'].value_counts()

        df = pd.DataFrame({'Quality of Service': normalized,
                           'Days': count}).reset_index()
        return df

    qos = qos_data(data)
    fig = px.bar(qos, x='plant_name', y='Quality of Service', color='health_state', hover_data=["Quality of Service", "Days"], orientation='v',
                 color_discrete_map=std_utils.health_state_colormap(), labels=std_utils.descriptions())
    fig.update_layout(xaxis_title=None)
    # ,config={'displayModeBar': False})

    # return pn.pane.Plotly(fig, sizing_mode='stretch_width')

    return fig

plot_qos(data)


In [ ]:

def plant_map(data):

    def plant_map_data(data):
        realtime = data['realtime']
        # TODO move !!!
        realtime['month_power_m'] = realtime['month_power'].apply(
            lambda d: f'{round(d /1000, 1)}')
        realtime['total_power_m'] = realtime['total_power'].apply(
            lambda d: f'{round(d /1000, 1)}')

        lastest = realtime[realtime.groupby(['plant_code'])['collect_time'].transform(
            max) == realtime['collect_time']]

        return pd.merge(lastest, data['plants'], on=['plant_code', 'plant_name'])

    map_data = plant_map_data(data)

    boundaries = {}
    with open('resources/geo/territoire.geojson') as f:
        boundaries = json.load(f)

    geojson = pdk.Layer(
        'GeoJsonLayer',
        boundaries,
        opacity=0.8,
        stroked=True,
        filled=False,
        extruded=False,
        get_line_color=[0, 0, 255],
        line_width_min_pixels=2,
    )

    pv = pdk.Layer(
        'ColumnLayer',
        data=map_data,
        get_position='[longitude,latitude]',
        radius=80,
        auto_highlight=True,
        elevation_scale=50,
        pickable=True,
        opacity=0.5,
        get_fill_color='(health_state == "Healthy") ? [0, 255, 0, 255] : ((health_state == "Faulty") ? [255, 0, 0, 255]: [128, 128, 128, 255])',
        get_elevation='capacity',
        extruded=True)

    tooltip = {
        "html": "<b>{plant_name}</b><br>{plant_addr}<br>Capacity: {capacity} kWc<br>Status: {health_state}<br>Last day: {day_power} kWh<br>Last month: {month_power_m} MWh<br> Total: {total_power_m} MWh",
        "style": {
            "background": "grey",
            "color": "white",
            "z-index": "10000"}
    }

    view_state = pdk.ViewState(
        latitude=45.71,
        longitude=5.87,
        zoom=11,
        min_zoom=9,
        max_zoom=15,
        pitch=50,
        bearing=0
    )

    deck = pdk.Deck(map_style='road', layers=[
                    geojson, pv], initial_view_state=view_state, tooltip=tooltip)
    return deck

plant_map(data)


In [ ]:
def alarm_trends(data):
    severities = data['alarms']['alarm_severity'].value_counts()

    numbers = []
    numbers.append({'name': 'Critical', 'icon': 'notification_important',
                    'value': severities.get('Critical', 0), 'unit': '', 'ratio': 100})
    numbers.append({'name': 'Major', 'icon': 'notifications',
                    'value': severities.get('Major', 0), 'unit': '', 'ratio': 100})
    numbers.append({'name': 'Minor', 'icon': 'error',
                    'value': severities.get('Minor', 0), 'unit': '', 'ratio': 100})
    numbers.append({'name': 'Warning', 'icon': 'warning',
                    'value': severities.get('Warning', 0), 'unit': '', 'ratio': 100})

    return numbers

alarm_trends(data)

In [ ]:

def power_trends(data):
    latest = latest_date(data)

    def power_per_plant(data, end, duration):
        in_range = data_in_range(data, end - duration, end)
        dl = in_range.groupby('plant_name')['inverter_power'].sum().to_frame(
            name='total').reset_index()
        return dl
    
    def power_per_plant_max(data):
        gp = data.groupby('plant_name')['inverter_power']
        dl = gp.max().to_frame(name='max').reset_index()
        return dl

    #pt.icon, pt.name, pt.value, pt.unit, pt.ratio
    numbers = []
    numbers.append({'name':'Capacity', 'icon':'solar_power', 'value':data['plants']['capacity'].sum(), 'unit':'kWp', 'ratio':100})
    
    ds = power_per_plant(data['hourly'], latest, datetime.timedelta(1))['total'].sum()
    dm = power_per_plant_max(data['daily'])['max'].sum()
    numbers.append({'name': 'Yesterday', 'icon': 'calendar_view_day', 'value': round(ds, 1), 'unit': 'kWh', 'ratio': round(100*ds/dm, 0)})

    ms = power_per_plant(data['hourly'], latest, datetime.timedelta(30))['total'].sum()
    mm = power_per_plant_max(data['monthly'])['max'].sum()
    numbers.append({'name':'Monthly', 'icon':'calendar_view_week', 'value':round(ms/1000, 1), 'unit':'MWh', 'ratio':round(100*ms/mm, 0)})

    ys = power_per_plant(data['hourly'], latest, datetime.timedelta(365))[
        'total'].sum()
    ym = power_per_plant_max(data['yearly'])['max'].sum()
    numbers.append({'name':'Yearly', 'icon':'calendar_view_month', 'value':round(ys/1000,1), 'unit':'MWh', 'ratio':round(100*ys/ym, 0)})

    lastest_realtime = latest_data(data['realtime'])
    numbers.append({'name':'Total', 'icon':'functions', 'value':round(lastest_realtime['total_power'].sum()/1000,1), 'unit':'MWh', 'ratio':None})
    
    return numbers

power_trends(data)

In [ ]:
def realtime_plot(data):
    realtime = data['realtime'][['collect_time', 'total_power', 'day_power', 'month_power', 'plant_name']]
    fig = px.area(realtime, x='collect_time', y='total_power',
                    color='plant_name',
                    hover_data=['total_power', 'day_power', 'month_power'],
                    color_discrete_sequence=px.colors.qualitative.D3,
                    labels=std_utils.descriptions())
    fig.update_traces(mode="markers+lines")
    fig.update_layout(hovermode="x")
    fig.update_layout(xaxis_title=None)  # , config={'displayModeBar': False})

    return fig

realtime_plot(data)

In [ ]:
def plot_power_rank(data, duration, divider, xtitle):
    latest = latest_date(data)
    def plot_power_rank_data(data, end, duration, divider):
        in_range = data_in_range(data, end - duration, end)
        dl = in_range.groupby('plant_name')['inverter_power'].sum().to_frame(
            name='total').reset_index()

        dl['total'] = dl['total'] / divider
        return dl
    
    df = plot_power_rank_data(data['hourly'], latest, duration, divider)

    fig = px.bar(df, x='total', y='plant_name', color='plant_name', orientation='h',
                 labels=std_utils.descriptions())
    fig.update_layout(xaxis_title=xtitle)
    fig.update_layout(yaxis_title=None)
    fig.update_layout(legend=None)

    return fig


def plot_power_rank_ytd(data):
    return plot_power_rank(data, datetime.timedelta(1), 1, 'Yield (kWh)')

def plot_power_rank_month(data):
    return plot_power_rank(data, datetime.timedelta(30), 1000, 'Yield (MWh)')

def plot_power_rank_year(data):
    return plot_power_rank(data, datetime.timedelta(365), 1000, 'Yield (MWh)')

def plot_power_rank_lifetime(data):
    return plot_power_rank(data, datetime.timedelta(365), 1000, 'Yield (MWh)')

plot_power_rank_month(data)

In [ ]:

def plot_power_ctrl(data, step, past):
    # h = data['collect_time'].max().to_pydatetime()
    h = datetime.datetime.now()
    #l = data['collect_time'].min()
    l = h-past if past else data['collect_time'].min()
    # l = min(data['collect_time'].min().to_pydatetime(), h - step)
    # l = h-step

    return {'min': l,
            'max': h,
            'rmin': max(l, h - past) if past else l,
            'rmax': h,
            'step': step}

def plot_power_data(data, begin, end):
    in_range = data_in_range(data, begin, end)
    return in_range[['collect_time', 'plant_name', 'inverter_power']]

def _plot_power(data, step, past, slider_format, tickformat, dtick):
    ctrl = plot_power_ctrl(data, step, past)

    data_range = plot_power_data(data, ctrl['min'], ctrl['max'])

    fig = px.bar(data_range, x='collect_time', y='inverter_power', color="plant_name", #template='plotly_dark',
                 barmode='stack', labels=std_utils.descriptions())
    fig.update_layout(xaxis=dict(tickformat=tickformat, dtick=dtick))
    fig.update_layout(autosize=True)

    return fig


def plot_power_hourly(data):
    return _plot_power(data['hourly'], datetime.timedelta(hours=1), datetime.timedelta(days=5), 'DD/MM/YYYY hh', '%Hh\n%d %b', '')

def plot_power_daily(data):
    return _plot_power(data['daily'], datetime.timedelta(days=1), datetime.timedelta(weeks=2*4), 'DD/MM/YYYY', '%d\n%b', '24*60*60*1000')

def plot_power_monthly(data):
    return _plot_power(data['monthly'], datetime.timedelta(weeks=4), datetime.timedelta(weeks=2*52), 'MM/YYYY', '%b\n%Y', 'M1')

def plot_power_yearly(data):
    return _plot_power(data['yearly'], datetime.timedelta(weeks=52), None, 'YYYY', '%Y', 'M12')


plot_power_yearly(data)


In [ ]:
import jinja2
import plotly.offline

template = """
<!DOCTYPE html>
<html lang="en">
<title>{{ title }}</title>
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="resources/css/w3.css">
<link rel="stylesheet" href="resources/css/w3-theme-green.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Ubuntu">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Material+Symbols+Outlined:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200">
<style>
body, h1, h2, h3, h4, h5, h6  {
  font-family: Ubuntu, sans-serif;
}
</style>

<!-- Plotly -->

<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

<!-- Tabs function -->
<script>
function openTab(classname, value) {
  var i;
  var x = document.getElementsByClassName(classname);
  for (i = 0; i < x.length; i++) {
    x[i].style.display = "none";
  }

  div = document.getElementById(value)
  div.style.display = "block";

  var divs = div.querySelectorAll("div.plotly-graph-div");
  for (var i = 0; i < divs.length; i++) {
    Plotly.relayout(divs[i], {autosize: true});
  }
}
</script>

{% macro big_number(icon, name, value, unit, ratio) -%}
    <div class="w3-container">
        <span class="w3-xxlarge">{{ value|e }}<span class="w3-medium">{{ unit|e }}</span></span>
        <span class="w3-right"><i class="material-symbols-outlined w3-xlarge w3-text-theme">{{icon}}</i></span>
        <div class="w3-light-grey">
            <div class="w3-theme w3-animate-opacity" style="height:3px;max-width:100%;width:{{ratio}}%;"></div>
        </div>
        <span class="">{{ name|e }}</span>
        {% if ratio %}
            <span class=w3-right>{{ratio}}%</span>
        {% endif %}
    </div>
{%- endmacro %}

{% macro alarm_number(icon, name, value) -%}
    <div class="w3-container">
        <span class="">{{ name|e }}</span>
        <span class="w3-right"><i class="material-symbols-outlined w3-xlarge w3-text-theme">{{icon}}</i></span>
        <div class="w3-xxlarge">{{ value|e }}</div>
    </div>
{%- endmacro %}

<body class="w3-margin">

    <!-- Power trends -->
    <div class="w3-margin-bottom">
        <div class="w3-cell-row w3-border w3-round w3-border-theme">
            {% for pt in ptrends %}
                <!-- Power trend -->
                <div class="w3-cell w3-mobile">
                    <div class="w3-cell-row">
                        <div class="w3-cell">
                            {{ big_number(pt.icon, pt.name, pt.value, pt.unit, pt.ratio) }}
                        </div>
                        {% if not loop.last %}
                            <div class="w3-cell w3-border-theme w3-border-right w3-hide-small"></div>
                        {% endif %}
                    </div>
                </div>
            {% endfor %}
        </div>
    </div>

    <!-- health -->
    <div class="w3-margin-top">
        <div class="w3-row-padding w3-stretch">
            <div class="w3-col m2">
                <div class="w3-round w3-border w3-border-theme">
                    <span class="w3-margin w3-text-theme">Alarms</span>
                    {% for pt in atrends %}
                        <!-- Alarm trend -->
                        {{ alarm_number(pt.icon, pt.name, pt.value) }}
                    {% endfor %}
                </div>
            </div>
            <div class="w3-col m3">
                <div class="w3-round w3-border w3-border-theme">
                    <span class="w3-margin w3-text-theme">Health</span>
                    {{plant_health}}
                </div>
            </div>
            <div class="w3-col m3">
                <div class="w3-round w3-border w3-border-theme">
                    <span class="w3-margin w3-text-theme">QoS</span>
                    {{qos}}
                </div>
            </div>
            <div class="w3-col m4">
                <div class="w3-round w3-border w3-border-theme"><iframe class="w3-border-0" style="width:100%;height:100%;" src="{{map}}"></iframe></div>
            </div>
        </div>
    </div>

    <!-- power -->
    <div class="w3-margin-top">
        <div class="w3-row-padding w3-stretch">
            <div class="w3-col m4"><div class="w3-round w3-border w3-border-theme">
                <span class="w3-margin w3-text-theme">Power ranking</span>
                {{power_ranking}}
            </div></div>
            <div class="w3-col m8"><div class="w3-round w3-border w3-border-theme">
                <span class="w3-margin w3-text-theme">Power</span>
                <div class="w3-bar w3-green">
                    <button class="w3-bar-item w3-button" onclick="openTab('power_tab', 'realtime')">Lifetime</button>
                    <button class="w3-bar-item w3-button" onclick="openTab('power_tab', 'hourly')">Hourly</button>
                    <button class="w3-bar-item w3-button" onclick="openTab('power_tab', 'daily')">Daily</button>
                    <button class="w3-bar-item w3-button" onclick="openTab('power_tab', 'monthly')">Monthly</button>
                    <button class="w3-bar-item w3-button" onclick="openTab('power_tab', 'yearly')">Yearly</button>
                </div>
                <div id="realtime" class="power_tab w3-animate-opacity" style="display:block;">{{realtime}}</div>
                <div id="hourly" class="power_tab w3-animate-opacity" style="display:none;">{{hourly}}</div>
                <div id="daily" class="power_tab w3-animate-opacity" style="display:none;">{{daily}}</div>
                <div id="monthly" class="power_tab w3-animate-opacity" style="display:none;">{{monthly}}</div>
                <div id="yearly" class="power_tab w3-animate-opacity" style="display:none;">{{yearly}}</div>
            </div></div>
        </div>
    </div>
</body>
</html> 
"""

j2_template = jinja2.Template(template)

def plotly_div(fig):
    config={'responsive': True, 'autosizable': True}
    return plotly.offline.plot(fig, include_plotlyjs=False, output_type='div', config=config)


def plotly_file(fig, filename):
    config = {'responsive': True, 'autosizable': True}
    plotly.offline.plot(fig, filename=filename, config=config, auto_open = False, image_width=10, image_height=10)
    return filename


# plant_map(data).to_html(as_string=True)
plant_map(data).to_html('map.html')

j2_params = {
    'title': 'SuperCV',
    'ptrends': power_trends(data),
    'atrends': alarm_trends(data),
    'plant_health': plotly_div(plot_qos(data)),
    'qos': plotly_div(plot_qos(data)),
    'map': "map.html",
    'power_ranking': plotly_div(plot_power_rank_month(data)),
    'realtime': plotly_div(realtime_plot(data)),
    'hourly': plotly_div(plot_power_hourly(data)),
    'daily': plotly_div(plot_power_daily(data)),
    'monthly': plotly_div(plot_power_monthly(data)),
    'yearly': plotly_div(plot_power_yearly(data)),
}

html = j2_template.render(j2_params, undefined=jinja2.StrictUndefined)

with open("jinja.html", "w") as fh:
    fh.write(html)